In [2]:
from transformers import MBartForConditionalGeneration

In [1]:
from indobenchmark import IndoNLGTokenizer

tokenizer = IndoNLGTokenizer.from_pretrained("indobenchmark/indobart-v2")

/home/haryoaw/mambaforge/envs/sensei/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model = MBartForConditionalGeneration.from_pretrained("haryoaw/stif-i-f-scratch")

In [ ]:
summary_ids = model.generate(inputs['input_ids'], decoder_input_ids = decoder_inputs['inputs_ids']  num_beams=4, max_length=5, early_stopping=True)

In [5]:
inp = 'min biaya gw dong, thx'

inp_to_model = tokenizer(inp, return_tensors='pt')['input_ids']
result = model.generate(
    inp_to_model, num_beams=8, max_length=100
)

In [8]:
tokenizer.decode(result[0])

'</s> admin , biaya saya , terima kasih . terima kasih biaya seperti admin . admin , admin . min . admin . saya biaya saya biaya biaya biaya admin . . . hahaha . . tolong biaya saya . . admin ? terima kasih , biaya seperti ini saya biaya min . min , biayax . . ya min . biaya admin , , biaya terima kasih min . . min min . ya admin . telepon admin , menggunakan menggunakan saya biaya nanti , biaya datang , admin , bukan bukan ? admin . kasih .</s>'

In [9]:
import pandas as pd

In [12]:
df = pd.read_csv("../data/stif_indo/test_with_pointing.csv")

In [13]:
from tqdm import tqdm

In [14]:
inf_result = []
for inf in tqdm(df.informal):
    inp_to_model = tokenizer(inf, return_tensors='pt')['input_ids']
    result = model.generate(
        inp_to_model, num_beams=10, max_length=100
    )
    inf_result.append(tokenizer.decode(result[0], skip_special_tokens=True))

100%|██████████| 363/363 [26:15<00:00,  4.34s/it]


In [83]:
inf_result

[' apakah masih ada konfirmasi lagi , admin ? terima kasih .',
 ' tolong notif yang itu . kemarin juga sampah sekali notif saya isinya gojek . hahaha .',
 ' halo , untuk keluhan telkomsel , silahkan konfirmasikan rekan kami di xuserx kak . - haski .',
 ' ini attempted fraud . potensi penipuan atas nama gojek . modus operandi : iming - iming hadiah . ada yang mau nge - flag xnumberx sebagai penipu ?',
 ' tolong , sudah xnumberx ribu . mengapa ? tolong . mengapa . tolong .',
 ' pengemudi xuserx dipersilahkan duduk di ruang tamu dan ada air mineral gelas yang disediakan untuk para " tamu - tamu tersebut .',
 ' admin , tolong informasikan resi id191432841909e dong . soalnya tujuan kiriman harusnya banjarbaru tapi dipindah ke banjarmasin . terima kasih .',
 ' yang lebih cepat dari gundala : habisnya kode promo grab .',
 ' sebelah kiri ? kita di twitter kak , buka instagram . hahaha .',
 ' laporan pulsa kesedot mulu sialaan . hahaha .',
 ' yang jadi provider seperti ini jadinya jadinya . jad

In [20]:
# use sacrebleu
from sacrebleu import corpus_bleu

## detokenize

corpus_bleu(inf_result, [df.formal.to_list()], lowercase=True)

BLEU = 4.50 12.6/6.5/3.3/1.5 (BP = 1.000 ratio = 5.776 hyp_len = 34074 ref_len = 5899)

In [17]:
from evaluate import load
bertscore = load("bertscore")
predictions = inf_result
references = df.formal.to_list()
results = bertscore.compute(predictions=predictions, references=references, lang="id")

In [18]:
predictions = inf_result
references = df.formal.to_list()
results = bertscore.compute(predictions=predictions, references=references, lang="id")

In [19]:
import numpy as np

np.mean(results['f1'])

0.6861076844296836